# Determine if SNP changes RSS of nearby m6A motif GGACU

## Step 1: Obtain mutation ID, ref and alt on `dbSNP`
Obtain mutation position according to its chromosome and position on genotype, and get responding `snpID` on `dbSNP`.

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
path = os.path.expanduser("~/Documents/m6A/")
exon_m6AQTL = f"{path}/Data/exon_m6AQTLs.txt"
output_exon_m6AQTL = f"{path}/Data/exon_m6AQTLs.bed"
intron_m6AQTL = f"{path}/Data/intron_m6AQTLs.txt"
output_intron_m6AQTL = f"{path}/Data/intron_m6AQTLs.bed"
output_dbSNP = f"{path}/Data/dbSNP/dbSNP.vcf.gz"

Import dbSNP and export the first five columns, only need to do it once.

In [2]:
# input_dbSNP = f"{path}/Data/dbSNP/common_all_20170710.vcf.gz"
# dbSNP = pd.read_table(input_dbSNP, header = 56, compression = "gzip", usecols = [0,1,2,3,4], low_memory = False)
# dbSNP.to_csv(output_dbSNP, compression = "gzip", sep = "\t", index = False)

Handle exon and intron m6AQTL data
1. sort by chromosome and pos
2. plus and minus 200 for each pos

In [3]:
def load_m6AQTL(data_path, n = 200, n_width = 1000):
    data = pd.read_table(data_path, header = 0)
    data = data.sort_values(by = ["chr", "pos"])
    data = data.set_index([[i for i in range(data.shape[0])]])
    data["strand"] = data.apply(lambda row: row["peakID"].split("_")[-1], axis = 1)
    data["gene_symbol"] = data.apply(lambda row: row["peakID"].split("_")[0], axis = 1)
    data["start"] = data.apply(lambda row: int(row["peakID"].split("_")[1]), axis = 1)
    data["end"] = data.apply(lambda row: int(row["peakID"].split("_")[2]), axis = 1)
    data["width"] = data.apply(lambda row: row["end"] - row["start"] + 1, axis = 1)
    data["pos1"] = data.apply(lambda row: row["pos"] - n, axis = 1)
    data["pos2"] = data.apply(lambda row: row["pos"] + n, axis = 1)
#     data["gap"] = data.apply(lambda row: row["start"] - row["pos"], axis = 1)
#     data["in"] = data.apply(lambda row: 1 if row["pos"] >= row["start"] and row["pos"] <= row["end"] else 0,axis=1)
    # only keep those width <= 1000
    data = data[data["width"] <= n_width]
    return data

In [4]:
intron_m6AQTL = load_m6AQTL(intron_m6AQTL, )
exon_m6AQTL = load_m6AQTL(exon_m6AQTL)

In [5]:
print (intron_m6AQTL.shape, exon_m6AQTL.shape)

(9312, 16) (2254, 16)


In [6]:
exon_m6AQTL[["chr", "pos", "peakID", "snpID", "start", "end", "width", "pos1", "pos2", "gene_symbol", "strand"]].head(5)

,chr,pos,peakID,snpID,start,end,width,pos1,pos2,gene_symbol,strand
1,chr1,943468,ISG15_949466_949516_+,rs3121567,949466,949516,51,943268,943668,ISG15,+
2,chr1,947538,ISG15_949466_949516_+,rs2465125,949466,949516,51,947338,947738,ISG15,+
3,chr1,948421,ISG15_949466_949516_+,rs113047134,949466,949516,51,948221,948621,ISG15,+
4,chr1,948846,ISG15_949466_949516_+,rs3841266,949466,949516,51,948646,949046,ISG15,+
5,chr1,948921,ISG15_949466_949516_+,rs15842,949466,949516,51,948721,949121,ISG15,+


In [7]:
intron_m6AQTL[["chr", "pos", "peakID", "snpID", "start", "end", "width", "pos1", "pos2", "gene_symbol", "strand"]].head(5)

,chr,pos,peakID,snpID,start,end,width,pos1,pos2,gene_symbol,strand
0,chr1,899937,AGRN_981788_981988_+,rs143296006,981788,981988,201,899737,900137,AGRN,+
1,chr1,899938,AGRN_981788_981988_+,rs147467971,981788,981988,201,899738,900138,AGRN,+
2,chr1,899942,AGRN_981788_981988_+,rs71509448,981788,981988,201,899742,900142,AGRN,+
3,chr1,943468,ISG15_949466_949516_+,rs3121567,949466,949516,51,943268,943668,ISG15,+
4,chr1,945612,ISG15_949466_949516_+,rs3121565,949466,949516,51,945412,945812,ISG15,+


In [8]:
# n1 = 1000
# # gap = intron_m6AQTL["gap"].tolist()
# intron_gap = intron_m6AQTL[(intron_m6AQTL["gap"] <= n1) & (intron_m6AQTL["gap"] >= -n1)]["gap"].tolist()
# plt.hist(intron_gap, bins = 30)
# plt.title("Histogram of gap in introns (the distance between SNP pos and start of peak)")
# plt.show()

In [9]:
# exon_gap = exon_m6AQTL[(exon_m6AQTL["gap"] <= n1) & (exon_m6AQTL["gap"] >= -n1)]["gap"].tolist()
# plt.hist(exon_gap, bins = 30)
# plt.title("Histogram of gap in exons (the distance between SNP pos and start of peak)")
# plt.show()

In [10]:
dbSNP = pd.read_table(output_dbSNP, compression = "gzip", header = 0, sep = "\t", low_memory = False)
dbSNP.head(5)

,#CHROM,POS,ID,REF,ALT
0,1,10177,rs367896724,A,AC
1,1,10352,rs555500075,T,TA
2,1,10352,rs145072688,T,TA
3,1,10616,rs376342519,CCGCCGTTGCAAAGGCGCGCCG,C
4,1,10642,rs558604819,G,A


In [11]:
def get_m6AQTL(data, output_path):
    res = pd.merge(data, dbSNP, how = "inner", left_on = ["pos", "snpID"], right_on = ["POS", "ID"])
    res = res[((res["start"] >= res["pos1"]) & (res["start"] <= res["pos2"])) 
              | ((res["end"] >= res["pos1"]) & (res["end"] <= res["pos2"])) 
              | ((res["start"] <= res["pos1"]) & (res["end"] >= res["pos2"]))]
    res = res.set_index([[i for i in range(res.shape[0])]])
    res[["chr", "pos1", "pos2", "gene_symbol", "width", "strand"]].to_csv(output_path, 
                                                                   index = False, header = False, sep = "\t")
    cols = ["#CHROM", "POS", "ID", "REF", "ALT", "strand", "start", "end", "width", "gene_symbol", "pos1", "pos2", 
            "beta", "FDR"]
    return res[cols]

In [12]:
intron_m6AQTL = get_m6AQTL(intron_m6AQTL, output_intron_m6AQTL)
exon_m6AQTL = get_m6AQTL(exon_m6AQTL, output_exon_m6AQTL)

In [13]:
print (intron_m6AQTL.shape, exon_m6AQTL.shape)

(188, 14) (378, 14)


In [14]:
exon_m6AQTL

,#CHROM,POS,ID,REF,ALT,strand,start,end,width,gene_symbol,pos1,pos2,beta,FDR
0,1,15756642,rs116128102,G,A,+,15756567,15756765,199,EFHD2,15756442,15756842,-0.456230,3.193890e-02
1,1,28212975,rs6564,C,T,+,28212824,28213122,299,THEMIS2,28212775,28213175,-0.163325,5.168226e-02
2,1,28213157,rs6565,T,C,+,28212824,28213122,299,THEMIS2,28212957,28213357,-0.163325,5.168226e-02
3,1,46810670,rs17361763,T,C,+,46810516,46810766,251,NSUN4,46810470,46810870,0.819328,2.263735e-07
4,1,46810842,rs6684274,C,T,+,46810516,46810766,251,NSUN4,46810642,46811042,0.819328,2.263735e-07
5,1,46830257,rs6683192,C,T,+,46830199,46830349,151,NSUN4,46830057,46830457,-0.696185,1.186266e-03
6,1,46830430,rs10252,G,A,+,46830199,46830349,151,NSUN4,46830230,46830630,-0.368553,8.971765e-02
7,1,46830447,rs12062,A,C,+,46830199,46830349,151,NSUN4,46830247,46830647,-0.696185,1.186266e-03
8,1,55316322,rs7374,A,G,-,55316022,55316670,649,DHCR24,55316122,55316522,0.200277,7.071380e-04
9,1,145441620,rs7211,T,"A,C",+,145441816,145441866,51,TXNIP,145441420,145441820,-0.350243,7.529982e-02


In [15]:
intron_m6AQTL

,#CHROM,POS,ID,REF,ALT,strand,start,end,width,gene_symbol,pos1,pos2,beta,FDR
0,1,46810670,rs17361763,T,C,+,46810516,46810766,251,NSUN4,46810470,46810870,0.819328,2.263735e-07
1,1,46810842,rs6684274,C,T,+,46810516,46810766,251,NSUN4,46810642,46811042,0.819328,2.263735e-07
2,1,155226233,rs115599747,T,C,-,155225843,155226040,198,SCAMP3,155226033,155226433,0.459673,3.113221e-02
3,1,155289545,rs11264361,T,"A,G",+,155289703,155290382,680,FDPS,155289345,155289745,0.327422,7.226476e-06
4,1,209952865,rs623360,C,G,+,209952704,209952952,249,TRAF3IP3,209952665,209953065,-0.326555,3.201475e-02
5,10,3178865,rs543,G,A,+,3178015,3178923,909,PFKP,3178665,3179065,-0.268602,9.222012e-02
6,10,5781628,rs2254067,T,G,+,5781827,5782326,500,FAM208B,5781428,5781828,0.352813,2.296571e-02
7,10,5781969,rs2797486,A,T,+,5781827,5782326,500,FAM208B,5781769,5782169,0.352813,2.296571e-02
8,10,91099593,rs34407818,A,G,+,91099654,91100202,549,IFIT3,91099393,91099793,0.250755,5.216162e-02
9,10,91100068,rs10887948,C,A,+,91099654,91100202,549,IFIT3,91099868,91100268,0.225057,1.910779e-03


In [16]:
list(set(exon_m6AQTL["gene_symbol"]))

['DCTD',
 'NSUN4',
 'ZSCAN16',
 'ZNF775',
 'TIMELESS',
 'FNIP2',
 'RAB15',
 'MED24',
 'REPS1',
 'MS4A1',
 'SSH1',
 'PPFIA1',
 'TRIP10',
 'TOMM22',
 'CSNK1G2',
 'ENTPD4',
 'HARBI1',
 'ZC3H7B',
 'HLA-C',
 'PIP4K2B',
 'DGKD',
 'MTG2',
 'C7orf50',
 'TRIM44',
 'RNF31',
 'ANKFY1',
 'ACLY',
 'RBM47',
 'EXOC4',
 'PFKP',
 'ACSL5',
 'DHCR24',
 'BCL9L',
 'USP40',
 'EMC3-AS1',
 'HLA-DOA',
 'SCD5',
 'PIGO',
 'SLC19A1',
 'ELAC2',
 'LY6E',
 'CENPF',
 'PPIL3',
 'DAP',
 'LMAN2',
 'ZZEF1',
 'IFIT3',
 'SAMD9L',
 'PGAP2',
 'C2CD2L',
 'SOGA1',
 'KCNJ12',
 'LOC642852',
 'TBX21',
 'MED29',
 'FAM168B',
 'MCFD2',
 'THEMIS2',
 'HSD17B4',
 'SNAI3-AS1',
 'FDPS',
 'LOC100133091',
 'TMEM55B',
 'RAI1',
 'GSTT1',
 'EZH1',
 'NARF',
 'MYO15B',
 'MYO19',
 'PHLDB3',
 'REPIN1',
 'EXOSC9',
 'CD81',
 'WBP1L',
 'EFHD2',
 'LMNB2',
 'SLC25A23',
 'SFXN5',
 'PRKXP1',
 'HLA-A',
 'PYCRL',
 'FAM188B',
 'TIMM10B',
 'HPS5',
 'ATP6V0D1',
 'CLASP1',
 'REXO4',
 'RTP4',
 'SDR39U1',
 'TANC1',
 'KLHL36',
 'RNASEK',
 'HLA-DRB1',
 'AKIRIN2',

In [17]:
from collections import Counter
Counter(exon_m6AQTL["width"])

Counter({51: 32,
         100: 9,
         101: 24,
         140: 1,
         147: 3,
         149: 4,
         150: 9,
         151: 20,
         171: 1,
         198: 1,
         199: 4,
         200: 37,
         201: 5,
         248: 3,
         249: 10,
         250: 13,
         251: 5,
         263: 1,
         291: 1,
         296: 1,
         297: 3,
         298: 1,
         299: 5,
         300: 13,
         301: 1,
         314: 11,
         345: 1,
         347: 8,
         348: 7,
         350: 1,
         351: 1,
         377: 1,
         379: 10,
         387: 1,
         396: 1,
         397: 1,
         398: 2,
         399: 1,
         400: 7,
         401: 3,
         406: 2,
         445: 1,
         447: 1,
         448: 1,
         449: 6,
         450: 3,
         486: 1,
         490: 1,
         495: 1,
         496: 2,
         497: 7,
         499: 7,
         500: 6,
         529: 1,
         531: 1,
         533: 3,
         545: 2,
         549: 5,
      

## Step 2: Use RNAsnp to obtain sequence

### Intersection of m6A peaks and 
Use "bedtools" in bash under the depository
`~/Documents/m6A/Data/metApeakFisher`
```
bedtools intersect -a ../intron_m6AQTLs.bed -b peaks.merged.bed -s > peak.merged.intron.m6AQTL.bed
bedtools intersect -a ../exon_m6AQTLs.bed -b peaks.merged.bed -s > peak.merged.exon.m6AQTL.bed
```

In [ ]:
n2 = 500
width = intron_m6AQTL[intron_m6AQTL["width"] <= n2]["width"].tolist()
plt.hist(width, bins = 25)
plt.title("Histogram of width")
plt.show()

# Step 2: 

@NTCNCCACCC:K00180:212:H7VCTBBXX:3:1101:21673:1033 1:N:0:NAATTCGT+AGGCTNTA
@NCGNTCAAGA:K00180:212:H7VCTBBXX:3:1101:21755:1033 1:N:0:NAATTCGT+AGGCTNTA
@NCTNCCCGAG:K00180:212:H7VCTBBXX:3:1101:21795:1033 1:N:0:NAATTCGT+AGGCTNTA
@NTCNTCCAAC:K00180:212:H7VCTBBXX:3:1101:22465:1033 1:N:0:NAATTCGT+AGGCTNTA

@<instrument>:<run number>:<flowcell ID>:<lane>:<tile>:<x-pos>:<y-pos> <read>:<is filtered>:<control number>:<sample number>